In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import deepRD
import deepRD.tools.analysisTools as analysisTools

In [ ]:
def read_numpy_arrays_from_file(file_path):
    arrays = []
    with open(file_path, 'r') as file:
        contents = file.read()

    # Split the file content based on square brackets [ ]
    array_strings = contents.split('[')[1:]

    # Convert each array string back to a numpy array and store it
    for array_string in array_strings:
        array = np.fromstring(array_string.replace(']', ''), sep=' ')
        arrays.append(array)

    return arrays

In [ ]:
# Load parameters from parameters file
simID = '0001'
localDataDirectory = os.environ.get('DATA') + 'openSystems/classicSmoluchowski_' + simID
parameterDictionary = analysisTools.readParameters(localDataDirectory + "/parameters_" + simID)
# Parameters for loading continuous trajectories from files (from original simulation)
numSimulations =  parameterDictionary['numSimulations']
D = parameterDictionary['D'] 
dt = parameterDictionary['dt']
stride = parameterDictionary['stride']
tfinal = parameterDictionary['tfinal']
kappa = parameterDictionary['kappa'] 
sigma = parameterDictionary['sigma']
R = parameterDictionary['R']
deltar = parameterDictionary['deltar']
cR = parameterDictionary['cR']
equilibrationSteps = parameterDictionary['equilibrationSteps']

In [ ]:
# Read files
file_path = localDataDirectory + '/classicSmol_nsims' + str(numSimulations) + '.xyz'
distancesArrays = read_numpy_arrays_from_file(file_path)

In [ ]:
# Calculate average concentration
nbins = 20
# Calculate values of middle points of histogram
hist, bins = np.histogram(distancesArrays[0], bins=nbins, range=(sigma,R))
rvals = np.zeros(len(bins)-1)
rvolumes = np.zeros(len(bins)-1)
for i in range(len(bins)-1):
    rvals[i] = (bins[i+1] + bins[i])/2.0
    rvolumes[i] = 4*np.pi*(bins[i+1]**3 - bins[i]**3)/3.0 # Approx alternative: 4*np.pi* rvals[i]**2*(bins[i+1]-bins[i])
# Calculate average concentration
concentrationHistogram = np.zeros(len(bins)-1)
for i in range(numSimulations):
    hist, bins = np.histogram(distancesArrays[i], bins=nbins, range=(sigma,R))
    concHist = hist/rvolumes
    concentrationHistogram += concHist
concentrationHistogram = concentrationHistogram/(numSimulations)

In [ ]:
fig, ax = plt.subplots()
bw=0.1

# Calculate analytical solution
kappa = 10
sigma = 1.0 
R = 10.0
cR = 1.0
rvals = np.arange(sigma,R,0.1)
kS = 4*np.pi*sigma*D
analyticCKRinf = cR*(1 - kappa*sigma/(kS + kappa)*(1/rvals))
analyticCK = cR*(1 - kappa*sigma/(kS + kappa*(1-sigma/R))*(1.0/rvals - 1.0/R))


#ax.plot(rvals, concentrationHistogram,'--k', label='simulation')
#ax.plot(rvals, analyticCKRinf,'-r', lw =3, label='benchmark Rinf')
ax.plot(rvals, analyticCK,'-k', label='benchmark')

ax.set_xlim((sigma, R))
ax.set_ylim((0, None))
ax.set_xlabel('radial distance')
ax.set_ylabel('Concentration')

#ax.legend(bbox_to_anchor=(0.6, 0., 0.5, 1.0), framealpha=1.0)

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
bw=0.1
concDensity = stats.kde.gaussian_kde(distancesArrays[0],bw)

distance = np.arange(sigma, R, 0.02)
#ax.plot(distance, concDensity(distance), '-k') # label='benchmark')
ax.plot(distance, concDensity(distance)/(4*np.pi*distance**2),'--k') # label='benchmark')

ax.set_xlim((sigma, R))
ax.set_ylim((0, None))
ax.set_xlabel('radial distance')
ax.set_ylabel('Concentration')

#ax.legend(bbox_to_anchor=(0.6, 0., 0.5, 1.0), framealpha=1.0)

plt.tight_layout()